In [83]:
import re
import regex
import pandas as pd
from datetime import datetime
import numpy as np

import eval7, pprint
from phevaluator import evaluate_cards
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
from poker.constants import Action, Position
from poker import handhistory

import matplotlib.pyplot as plt
plt.style.use('dark_background') #стиль графиков

import sqlite3

import sentry_sdk

In [400]:
%%time

# запрос из базы
conn = sqlite3.connect(r'C:\Users\artio\Documents\Holdem Manager 3\Databases\Bot_Praktik_2023.hmdb')
cur = conn.cursor() # создаём курсор для получения не всей базы а чтения по строчно
data_r = ''
for i, line in enumerate(cur.execute('select * from handhistories;')): # limit 10
    data_r = data_r + line[3]
    if i > 20:
        break

iterr = re.findall("PokerStars", data_r).count('PokerStars') # задаём колличесвто итераций
print(iterr)

data = ""
iter_SHOW = 0

for i in range(iterr+1): # фильтруем раздачи с ШД и без
    if data_r.split("PokerStars")[i].count('SHOW DOWN'):
        data = data + data_r.split("PokerStars")[i] + "\n\n\n"
        iter_SHOW += 1
    else:
        continue
print(iter_SHOW) 

# -------------------------  ПРЕФЛОП  ------------------------------

def preflop_acti(self):
    self = preflop_#re.search("HOLE CARDS \*{3}(.*)\*{3} FLOP", self, re.DOTALL)
    action_preflop_raises = preflop_.group(1).count('raises')    
    action_preflop_calls = preflop_.group(1).count('calls')
    action_preflop_allin = preflop_.group(1).count('all-in')
    return action_preflop_raises, action_preflop_calls, action_preflop_allin

def street_raise_sum(self):
    street_ = self.group(1)    
    matches = re.findall(r'(?<=raises) (\d*) to (\d*)',street_)  
    n = 0
    sum_rais = []
    for i in enumerate(matches):
        rais = int(matches[n][1])
        sum_rais.append(rais)
        n += 1
    return sum(sum_rais)

def street_calls_sum(self):
    street_ = self.group(0)    
    matches = re.findall(r'(?<=calls) (\d*)',street_)  
    n = 0
    sum_calls = []
    for i in enumerate(matches):
        calls = int(matches[n])
        sum_calls.append(calls)
        n += 1
    return sum(sum_calls)

# -------------------------  ФЛОП  ------------------------------

def flop_acti(self):
    flop_ = self
    action_flop_bets = flop_.group(1).count('bets') # колличесвто
    action_flop_raises= flop_.group(1).count('raises')
    action_flop_calls = flop_.group(1).count('calls')
    action_flop_allin = flop_.group(1).count('all-in') 
    return action_flop_bets, action_flop_raises, action_flop_calls, action_flop_allin

def street_bets(self):
    street_ = self.group(1)
    matches = re.findall(r'(?<=bets) (\d*)',street_)
    if matches:
        return int(matches[0])
    else:
        return 0

# -------------------------  ТЁРН  ------------------------------

def turn_acti(self):
    turn_ = self
    action_turn_bets = turn_.group(1).count('bets')
    action_turn_raises= turn_.group(1).count('raises')
    action_turn_allin = turn_.group(1).count('all-in')
    action_turn_calls = turn_.group(1).count('calls')
    return action_turn_bets, action_turn_raises, action_turn_calls, action_turn_allin

# -------------------------  РИВЕР  ------------------------------

def river_acti(self):
    river_ = self
    action_river_bets = river_.group(1).count('bets')
    action_river_raises= river_.group(1).count('raises')
    action_river_allin = river_.group(1).count('all-in')
    action_river_calls = river_.group(1).count('calls')
    return action_river_bets, action_river_raises, action_river_calls, action_river_allin

#----ПРЕФЛОП------
preflop_raisess = []
preflop_callss = [] 
preflop_allins = []
preflop_raisess_sums =[]
preflop_calls_sums =[]

#----ФЛОП------
flop_betss = []  
flop_raisess = [] 
flop_callss = []
flop_allins = []
flop_chip_betss = []  
flop_chip_callss = []
flop_chip_raisess = [] 

#------ТЁРН--------
turn_betss = []  
turn_raisess = [] 
turn_callss = []
turn_allins = []
turn_chip_betss = []  
turn_chip_raisess = [] 
turn_chip_callss = []

#-------РИВЕР---------
river_betss = []  
river_raisess = [] 
river_callss = []
river_allins = []
river_chip_betss = []  
river_chip_raisess = [] 
river_chip_callss = []


for y in range(iter_SHOW):
    data_n = data.split("\r\n\n\n\n")[y] # разбивает сплошной текст на раздачи
    # print(repr(data_n))
# -------------------------  ПРЕФЛОП  ------------------------

    preflop_ = re.search("HOLE CARDS \*{3}(.*)\*{3} FLOP", data_n, re.DOTALL)
    
    preflop_raisess.append(preflop_acti(preflop_)[0])
    preflop_callss.append(preflop_acti(preflop_)[1])
    preflop_allins.append(preflop_acti(preflop_)[2])
    preflop_raisess_sums.append(street_raise_sum(preflop_))
    preflop_calls_sums.append(street_calls_sum(preflop_))
    
# ------------------------- ФЛОП  ------------------------

    flop_ = re.search("FLOP \*{3} \[.*\](.*)\*{3} TU", data_n, re.DOTALL)

    flop_betss.append(flop_acti(flop_)[0])
    flop_raisess.append(flop_acti(flop_)[1])
    flop_callss.append(flop_acti(flop_)[2])
    flop_allins.append(flop_acti(flop_)[3])    
    flop_chip_betss.append(street_bets(flop_)) 
    flop_chip_callss.append(street_calls_sum(flop_))
    flop_chip_raisess.append(street_raise_sum(flop_))

# -------------------------  ТЁРН   ------------------------

    turn_ = re.search("TURN \*{3} \[.*\] \[.*\](.*)\*{3} RI", data_n, re.DOTALL)
    
    turn_betss.append(turn_acti(turn_)[0])
    turn_raisess.append(turn_acti(turn_)[1])
    turn_callss.append(turn_acti(turn_)[2])
    turn_allins.append(turn_acti(turn_)[3])
    turn_chip_betss.append(street_bets(turn_))
    turn_chip_callss.append(street_calls_sum(turn_))
    turn_chip_raisess.append(street_raise_sum(turn_))

# -------------------------  РИВЕР  ------------------------

    river_ = re.search("RIVER \*{3} \[.*\] \[.*\](.*)\*{3} SHOW", data_n, re.DOTALL)
 
    river_betss.append(turn_acti(river_)[0])
    river_raisess.append(turn_acti(river_)[1])
    river_callss.append(turn_acti(river_)[2])
    river_allins.append(turn_acti(river_)[3])
    river_chip_betss.append(street_bets(river_))
    river_chip_callss.append(street_calls_sum(river_))
    river_chip_raisess.append(street_raise_sum(river_))    

# print(river_betss)
# print(river_raisess) 
# print(river_callss)
# print(river_allins)
# print(river_chip_betss)
# print(river_chip_raisess)
# print(river_chip_callss)


22
10
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 271175, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 78507, 0, 0, 0, 0, 0]
CPU times: total: 15.6 ms
Wall time: 7.99 ms


In [305]:
ante_ = int(re.findall('ante (\d*)', data)[0])
ante_

250

In [396]:
street_ = river_.group(1)
print(repr(street_))
matches = re.findall(r'(?<=bets) (\d*)',street_)
if matches:
    print(int(matches[0]))
else:
    print(0)

'\r\n'
0
